Import Libraries

In [1]:
import gurobipy as gp
from gurobipy import *
import math
import pandas as pd

# Input Parameters

In [2]:
Distance_Matrix = pd.read_excel("C:/Users/Palak/OneDrive/Desktop/ISP_Dataset.xlsx", sheet_name="Distance Matrix")
Transportation_Cost_Matrix = pd.read_excel("C:/Users/Palak/OneDrive/Desktop/ISP_Dataset.xlsx", sheet_name="Transportation cost matrix")
VS_capacity = pd.read_excel("C:/Users/Palak/OneDrive/Desktop/ISP_Dataset.xlsx", sheet_name="Capacity of DVS")
DG_demand = pd.read_excel("C:/Users/Palak/OneDrive/Desktop/ISP_Dataset.xlsx", sheet_name="Demand of PHC")
M_capacity = pd.read_excel("C:/Users/Palak/OneDrive/Desktop/ISP_Dataset.xlsx", sheet_name="Capacity of Manufacturers")

In [3]:
# Number of Maufacturer and suppliers
M = 7
# Number of District Vaccince Store
V = 20
# Number of Primary Health Centre
J = 280
# Number of DVS to be allocated
P = 12
#Holding cost per vaccine per year
CH = 20

In [4]:
# 2D-list for Travel distance matrix between each V and J
dij = Distance_Matrix.iloc[:, 1:].values.tolist()

#2D-list for Transportation cost per order of vaccines provided by Manufacturers
Cgi = Transportation_Cost_Matrix.iloc[:, 1:].values.tolist()

# list for capacity of each District vaccine store
Si = VS_capacity['Capacity'].tolist()

# list for demand for each PHC
qj = DG_demand['Demand'].tolist()

# list for Capacity of Manufacturers
Eg = M_capacity['Capacity'].tolist()

# Stage-1  

(To establish network between Primary Health Center(PHC) and District Vaccine Store(DVS), & also to get demand of each District Vaccine  store)

In [5]:
# Create a new LP model
M1 = gp.Model("min_travel_distance")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-01


In [6]:
# Add binary variables to the model
xij = M1.addVars(V, J, vtype=GRB.BINARY, name='xij')
Xi  = M1.addVars(V, vtype=GRB.BINARY, name="Xi")

In [7]:
# Set the objective function
M1.setObjective(gp.quicksum((qj[j] * xij[i, j] * dij[i][j]) for j in range(J) for i in range(V))/sum(qj), GRB.MINIMIZE)

Add constraints to the model

In [8]:
# Every PHC must be fully served
M1.addConstrs((gp.quicksum(xij[i, j] for i in range(V)) == 1) for j in range(J))

# There is enough capacity of DVS to serve allocated PHC
M1.addConstrs((gp.quicksum(xij[i, j] * qj[j] for j in range(J)) <= Si[i]) for i in range(V))
M1.update()

In [9]:
# Allowable number of DVS are P.
M1.addConstr((gp.quicksum(Xi[i] for i in range(V)) == P))
M1.update()

In [10]:
#Assignment of DVS should not be made to a PHC untill a warehouse is opened in that location
M1.addConstrs((xij[i, j] <= Xi[i]) for i in range(V) for j in range(J))
M1.update()

In [11]:
# Optimize the model
M1.optimize()
M1.printAttr('X')

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 5901 rows, 5620 columns and 22420 nonzeros
Model fingerprint: 0xccbdffe2
Variable types: 0 continuous, 5620 integer (5620 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+06]
  Objective range  [3e-03, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve time: 0.03s
Presolved: 5901 rows, 5620 columns, 22420 nonzeros
Variable types: 0 continuous, 5620 integer (5620 binary)

Root relaxation: objective 5.115495e+01, 539 iterations, 0.02 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   51.15495    0   50          -   51.15495      -     -    0s
H    0     0           

  xij[1,104]            1 
  xij[1,106]            1 
  xij[1,107]            1 
  xij[1,108]            1 
  xij[1,109]            1 
  xij[1,113]            1 
  xij[1,114]            1 
  xij[1,116]            1 
  xij[1,118]            1 
  xij[1,121]            1 
  xij[1,123]            1 
  xij[1,124]            1 
  xij[1,125]            1 
  xij[1,129]            1 
  xij[1,130]            1 
  xij[1,131]            1 
  xij[1,134]            1 
  xij[1,135]            1 
  xij[1,136]            1 
  xij[1,137]            1 
  xij[1,138]            1 
  xij[1,139]            1 
  xij[1,144]            1 
  xij[1,179]            1 
  xij[1,182]            1 
  xij[1,190]            1 
  xij[1,270]            1 
   xij[2,10]            1 
   xij[2,46]            1 
   xij[2,53]            1 
   xij[2,54]            1 
   xij[2,55]            1 
   xij[2,56]            1 
   xij[2,93]            1 
   xij[2,97]            1 
  xij[2,105]            1 
  xij[2,128]            1 
 

In [32]:
DVS = VS_capacity['District Vaccine Store'].tolist() 
PHC = DG_demand['Primary Health Centers'].tolist()

In [34]:
Arcs2={}
list3=[]
for i in range(V):
    for j in range(J):
        if xij[i,j].X == 1:
            list3.append(PHC[j])
    if len(list3)!=0:
        Arcs2[str(DVS[i])]=list3
        list3=[]
    else:
        continue
print(Arcs2)


{'Bankura': ['Frezergunge PHC', 'East Gabberia PHC', 'Dakshin Radhanagar PHC', 'Nalgara PHC', 'Betamaheshpur P.H.C.', 'Kelomal P.H.C.', 'Jamalpur PHC', 'Ramkishorepur PHC', 'Simulia P.H.C.', 'Bibhisanpur P.H.C.', 'Nayaput PHC', 'Chatla P.H.C.', 'Natsal P.H.C.', 'Amdabad P.H.C.', 'Boyal P.H.C.', 'Argoal P.H.C.', 'Chandanpur P.H.C.', 'Gobra P.H.C.', 'Pichhaboni P.H.C.', 'Uttar Dhalhara P.H.C.', 'Bhagwanpur RH', 'Bhupatinagar Mughberia RH', 'Basantia RH', 'Uttar Mechogram RH', 'Janubasan RH', 'Anantapur RH', 'Contai SD Hospital', 'Egra SD Hospital', 'Brajaballavpur PHC', 'Bari BPHC', 'Balitora (Raghunthpur) BPHC', 'Puruliya Jail Hospital', 'Puruliya Mental Hospital', 'Puruliya Police Hospital', 'Arsha PHC', 'Kantadih PHC', 'Koreng PHC', 'Bamundiha PHC', 'Manbajar RH', 'Puncha RH', 'Kulberia P.H.C.', 'Bindol PHC'], 'Bardhaman': ['Kasba Police Hospital', 'Baghajatin S G Hospital', 'Mohammadpur P.H.C.', 'Haridevpur PHC', 'Gokarni PHC', 'Belpukur PHC', 'Gokhuri P.H.C.', 'Barsundra P.H.C.', 'P

In [12]:
# Define the demand for each DVS
D = [gp.quicksum(xij[i, j] * qj[j] for j in range(J)) for i in range(V)]

In [13]:
# Print the demand at each vaccination station
#for i in range(V):
#    print(f"Demand at vaccination station {i}: {D[i].getValue()}")

# Stage -2

(To establish network between District Vaccine Store(DVS) and Manufacturers )

In [14]:
M2 = gp.Model("min_Transportation_&_Holding_cost")

In [15]:
# Calculate Economic order quantity of vaccine 'O'
O = [2 * gp.quicksum(xij[i, j] * qj[j] * Cgi[g][i] for j in range(J) for g in range(M)) / CH for i in range(V)]

O_values = [math.sqrt(O[i].getValue()) for i in range(V)]
D_values = [D[i].getValue() for i in range(V)]                  

In [16]:
# Add binary variables to the model
ugi = M2.addVars(M, V, vtype=GRB.BINARY, name='Ugi')

In [17]:
# Set the objective function

M2.setObjective(gp.quicksum((Cgi[g][i]* ugi[g,i] ) * (D_values[i]/O_values[i] if O_values[i] != 0 else 0) for i in range(V) for g in range(M)) + gp.quicksum(CH *O_values[i]/2 for i in range(V)), GRB.MINIMIZE)


Add constraints to the model

In [18]:
# DVS should be assigned to atleast one cold storage point
M2.addConstrs(gp.quicksum(ugi[g,i] for g in range(M)) >= 1 for i in range(V));

# Each Manufacturer should supply vaccines to atleast one DVS
M2.addConstrs(gp.quicksum(ugi[g,i] for i in range(V)) >= 1 for g in range(M));

# All Manufacturer points are sufficient to fulfill the demand for vaccines at the DVS
M2.addConstrs(gp.quicksum(ugi[g,i]*Eg[g] for g in range(M))>=D_values[i] for i in range(V));

M2.update()

In [19]:
M2.optimize()
M2.printAttr('X')

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 47 rows, 140 columns and 420 nonzeros
Model fingerprint: 0x3ea66525
Variable types: 0 continuous, 140 integer (140 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+07]
  Objective range  [5e+04, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Found heuristic solution: objective 1.593644e+07
Presolve removed 46 rows and 134 columns
Presolve time: 0.01s
Presolved: 1 rows, 6 columns, 6 nonzeros
Found heuristic solution: objective 1.554586e+07
Variable types: 0 continuous, 6 integer (6 binary)

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 2: 1.55459e+07 1.59364e+07 

Optimal solution found (tolerance 1.00e-04)
Best objec